# TP6 - corps binaires et applications

<span class='alert-info'> Répondez aux questions dans un notebook séparé que vous rendrez au format <strong> NOM1_NOM2_tp6.ipynb </strong> </span>

## Partie 1 - un anneau qui a du caractère

On va travailler dans un **anneau** (structure algébrique où $+, -, \times$ sont autorisées) **fini** à $64$ éléments noté $(\mathbb{F}_{64}, +, \times)$. Les éléments de $\mathbb{F}_{64}$ seront informatiquement des instances de la classe ```F``` implémentée ci-dessous. 

In [23]:
class F(SageObject):
    
    # names for the 64 elements
    # static variables 
    names = "01abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ !?-'#$%&@"
    
    # create F object associated to character val 
    def __init__(self, val):
        
        try:
            # id = integer to identify the ring element 
            self.id = Integer(F.names.index(str(val)))
            
        except ValueError:
            # val is not found in names 
            raise ValueError("Element %s unknown" % val)
            
    # F1 == F2 if F1 and F2 have the same id 
    def __eq__(self, other):
        
        return self.id == other.id
        
    def __ne__(self, other):
        
        return not (self == other)
       
    # special method to represent an instance of F 
    def __repr__(self):
        
        return F.names[self.id]
    
    # resultat of F1 + F2
    def __add__(self, other):
        
        r = F(self)
        r += other
        return r
    
    # result of F1 += F2
    def __iadd__(self, other):
        
        # add two objects = bitwise addition on their id 
        self.id = self.id.__xor__(other.id)
        return self
    
    # method that is used to perform F1 * F2 
    def shift(self):
        
        # append a 0 in the binary representation of self.id 
        # tmp is an integer coded with 7 bits 
        tmp = self.id << 1
        
        # res stores bits associated to 2^6 to 2^0 
        res = tmp % 64
        
        # tmp // 64 stores the bit associated to 2^7  
        # if the bit is 1...
        if tmp // 64:
            
            # 1 000 000 is represented by 0 011 011 (why?)
            # perform bitwise addition 011 011 + res  
            res = res.__xor__(27) 
            
        self.id = res
    
    # result of F1 * F2 
    # magic multiplication 
    def __mul__(self, other):
        
        res = F(0)
        tmp = F(other)
        
        # F1 is represented as a binary word 
        # when a bit is 0, do nothing 
        # when a bit is 1, add a shifted version of F(other)
        for b in self.id.bits():
            if b:
                res += tmp
            tmp.shift()
            
        return res
        
    # resultat of F*F*...*F
    def __pow__(self, exp):
        
        res = F(1)
        
        for i in range(exp):
            res *= self
        
        return res
        
F.elements = [F(i) for i in F.names]

Les éléments de $\mathbb{F}_{64}$ sont les 64 caractères (ou *symboles*) suivants:

In [24]:
print(F.elements)

[0, 1, a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, q, r, s, t, u, v, w, x, y, z, A, B, C, D, E, F, G, H, I, J, K, L, M, N, O, P, Q, R, S, T, U, V, W, X, Y, Z,  , !, ?, -, ', #, $, %, &, @]


On crée facilement les *objets* associés à chaque élément. Par exemple, pour créer l'objet associé à l'élément $a$: 

In [25]:
a = F("a")
a

a

Deux opérations de base sont implémentées : une **addition**

In [26]:
c = F("c")
a + c

e

et une **multiplication**

In [27]:
a * c

g

Notez que l'anneau $(\mathbb{F}_{64}, +, \times)$ n'est **pas** isomorphe à $(\mathbb{Z}/64 \mathbb{Z}, +, \times)$. On peut par exemple vérifier que $\alpha + \alpha = 0$ pour tout $\alpha \in \mathbb{F}_{64}$ (alors que dans $\mathbb{Z}/64\mathbb{Z}$, chaque élément n'est pas son propre opposé) 

 Dans l'anneau $\mathbb{F}_{64}$, chaque élément est son propre opposé parce que l'opération d'addition est définie comme une addition bit à bit (aussi connue sous le nom d'opération XOR) sur les identifiants des éléments.

Dans l'anneau $\mathbb{Z}/64\mathbb{Z}$, chaque élément n'est pas son propre opposé car l'opération d'addition est définie comme une addition modulaire sur les entiers de $0$ à $63$.

Plus précisément, si $a$ est un élément de $\mathbb{Z}/64\mathbb{Z}$, alors son opposé est l'élément $-a$ tel que $a + (-a) = 0$ dans l'anneau. Cependant, dans cet anneau, l'addition est définie modulo $64$, ce qui signifie que l'opposé de $a$ est en réalité l'élément $64-a$ plutôt que $-a$.

Par exemple, dans cet anneau, l'élément $1$ n'est pas son propre opposé car son opposé est l'élément $63$, et $1+63=64$, qui est congruent à $0$ modulo $64$. De même, l'élément $32$ n'est pas son propre opposé car son opposé est l'élément $32$ (car $32+32=64$), qui est égal à $0$ dans cet anneau.

In [28]:
alpha = choice(F.elements)  # random element, run again for another one

print("alpha = %s, alpha + alpha = %s" % (alpha, alpha + alpha))

alpha = c, alpha + alpha = 0


### a) C'est un anneau commutatif

<div class='alert-danger'> <strong> Alerte à la question! </strong> </div>

Vérifier explicitement (par *force brute*) que **tous** les axiomes d'anneau commutatif sont satisfaits pour cette structure algébrique.

Pour vérifier que tous les axiomes d'un anneau commutatif sont satisfaits pour cette structure algébrique, nous devons vérifier les propriétés suivantes :

1 - Associativité de l'addition: $$(a + b) + c = a + (b + c)$$
2 - Commutativité de l'addition: $$a + b = b + a$$
3 - Existence de l'élément neutre de l'addition: $$a + 0 = a$$
4 - Existence de l'opposé pour l'addition: $$a + (-a) = 0$$
5 - Associativité de la multiplication: $$(a * b) * c = a * (b * c)$$
6 - Commutativité de la multiplication: $$a * b = b * a$$
7 - Existence de l'élément neutre de la multiplication: $$a * 1 = a$$
8 - Distributivité de la multiplication par rapport à l'addition: $$a * (b + c) = (a * b) + (a * c)$$

In [29]:
# Vérifier l'associativité de l'addition
for a in F.elements:
    for b in F.elements:
        for c in F.elements:
            if (a + b) + c != a + (b + c):
                print(f"Addition non associative: ({a} + {b}) + {c} != {a} + ({b} + {c})")

# Vérifier la commutativité de l'addition
for a in F.elements:
    for b in F.elements:
        if a + b != b + a:
            print(f"Addition non commutative: {a} + {b} != {b} + {a}")

# Vérifier l'élément neutre de l'addition et l'existence de l'opposé pour l'addition
zero = F(0)
for a in F.elements:
    if a + zero != a:
        print(f"Élément neutre de l'addition incorrect: {a} + {zero} != {a}")
    if a + a != zero:
        print(f"Opposé pour l'addition non trouvé: {a} + {a} != {zero}")

# Vérifier l'associativité de la multiplication
for a in F.elements:
    for b in F.elements:
        for c in F.elements:
            if (a * b) * c != a * (b * c):
                print(f"Multiplication non associative: ({a} * {b}) * {c} != {a} * ({b} * {c})")

# Vérifier la commutativité de la multiplication
for a in F.elements:
    for b in F.elements:
        if a * b != b * a:
            print(f"Multiplication non commutative: {a} * {b} != {b} * {a}")

# Vérifier l'élément neutre de la multiplication
one = F(1)
for a in F.elements:
    if a * one != a:
        print(f"Élément neutre de la multiplication incorrect: {a} * {one} != {a}")

# Vérifier la distribution de la multiplication sur l'addition
for a in F.elements:
    for b in F.elements:
        for c in F.elements:
            if a * (b + c) != a * b + a * c:
                print(f"Distribution de la multiplication sur l'addition incorrecte: {a} * ({b} + {c}) != {a} * {b} + {a} * {c}")

# Afficher un message de confirmation si toutes les propriétés sont vérifiées avec succès
print("Toutes les propriétés des axiomes d'un anneau commutatif ont été vérifiées avec succès pour cette structure algébrique.")


Toutes les propriétés des axiomes d'un anneau commutatif ont été vérifiées avec succès pour cette structure algébrique.


### b) Inverse d'un élément

Il est aisé de vérifier qu'un élément est inversible si on nous fournit un candidat pour son inverse. Par exemple: le calcul ci-dessous suffit à convaincre que l'élément $\mathtt{a}$ est inversible et que $\mathtt{a}^{-1} = \mathtt{R}$.

In [30]:
F("a") * F("R")

1

<div class='alert-danger'> <strong> Alerte à la question! </strong> </div>

Soit $m$ la somme des numéros des mois de naissance des membres du binôme et $\zeta$ l'élément `F.elements[m]` de $\mathbb{F}_{64}$. Vérifiez que $\zeta$ est inversible en recherchant par force brute son inverse parmi les éléments de $\mathbb{F}_{64}$.

In [31]:
m = 25  # somme des mois de naissance des membres du trinôme
zeta = F.elements[m]

zeta_inverse = None
for candidate in F.elements:
    if zeta * candidate == F(1):
        zeta_inverse = candidate
        break

if zeta_inverse is not None:
    print(f"L'inverse de {zeta} est {zeta_inverse}  :  {zeta} * {zeta_inverse} = 1")
else:
    print(f"{zeta} n'est pas inversible dans l'anneau")


L'inverse de x est W  :  x * W = 1


Il n'est pas nécessaire de vérifier la commutativité de l'inverse. Étant donné que la multiplication dans l'anneau $\mathbb{F}_{64}$ est déjà définie comme étant commutative (nous l'avons vérifié précédemment), il suffit de vérifier que $\zeta * \zeta^{-1} = 1$. 

### c) Structure multiplicative

Vous l'aurez peut-être deviné : tous les éléments non nuls de $\mathbf{F}$ sont inversibles (il s'agit donc d'un *corps fini* à 64 éléments). 

<div class='alert-danger'> <strong> Alerte à la question! </strong> </div>

- Déterminer l'ordre multiplicatif de l'élément $\mathtt{a}$
- En déduire que tous les éléments de $\mathbb{F}_{64}$ (sauf le neutre additif) sont inversibles. 
- En partant de $\mathtt{a}^{-1} = \mathtt{R}$, déterminer astucieusement (donc pas par force brute) l'inverse de $\zeta$

1) L'ordre multiplicatif d'un élément $\mathtt{a}$ d'un anneau commutatif est défini comme le plus petit entier positif $n$ tel que $\mathtt{a}^n = 1$, où $1$ est l'élément neutre de la multiplication dans l'anneau. 

In [32]:
a = F("a")
n = 1
while a ** n != F(1):
    n += 1

print(f"L'ordre multiplicatif de a est {n}")
print("Donc : a^63 = ", a^63)

L'ordre multiplicatif de a est 63
Donc : a^63 =  1


2) Lorsque nous avons calculé l'ordre multiplicatif de $\mathtt{a}$, nous avons trouvé le plus petit entier positif $n$ tel que $\mathtt{a}^n = 1$. Cela signifie que $\mathtt{a}^{n-1}$ est l'inverse de $\mathtt{a}$, car $\mathtt{a} * \mathtt{a}^{n-1} = \mathtt{a}^n = 1$.

Maintenant, nous savons que $\mathtt{a}$ engendre le groupe multiplicatif. Cela signifie que tous les éléments non nuls de $\mathbb{F}{64}$ peuvent être exprimés sous la forme $\mathtt{a}^k$ pour un certain entier $k$. Donc, pour chaque élément non nul $\zeta$ de $\mathbb{F}{64}$, il existe un entier $k$ tel que $\zeta = \mathtt{a}^k$.

En utilisant la propriété que $\mathtt{a}^{n-1}$ est l'inverse de $\mathtt{a}$, nous pouvons calculer l'inverse de $\zeta$ comme suit: $\zeta^{-1} = (\mathtt{a}^k)^{-1} = \mathtt{a}^{-k}$. Puisque nous pouvons calculer l'inverse de chaque élément non nul $\zeta$ de cette manière, cela signifie que tous les éléments non nuls de $\mathbb{F}_{64}$ sont inversibles.

3) Nous savons déjà que tous les éléments de $\mathbb{F}{64}$ (à l'exception du neutre additif) sont inversibles et que $\mathtt{a}$ engendre le groupe multiplicatif. Cela signifie que chaque élément $\zeta$ de $\mathbb{F}{64}$ peut être exprimé sous la forme $\zeta = \mathtt{a}^k$ pour un certain entier $k$.

Le but est de déterminer l'inverse de $\zeta$ sans utiliser la force brute. Pour ce faire, nous utilisons la relation suivante :

\begin{equation}
\zeta^{-1} = (\mathtt{a}^k)^{-1} = \mathtt{a}^{-k}
\end{equation}

L'inverse de $\zeta$ est simplement $\mathtt{a}^{-k}$. Pour calculer $\mathtt{a}^{-k}$, nous avons besoin de connaître l'inverse de $\mathtt{a}$, qui est $\mathtt{R}$ :

\begin{equation}
\mathtt{a}^{-1} = \mathtt{R}
\end{equation}

En utilisant cette information, nous pouvons calculer $\mathtt{a}^{-k}$ comme suit :

\begin{equation}
\mathtt{a}^{-k} = (\mathtt{a}^{-1})^k = \mathtt{R}^k
\end{equation}

Maintenant, divisons le processus en deux étapes :

Trouver l'entier $k$ tel que $\zeta = \mathtt{a}^k$.
Calculer l'inverse de $\zeta$ en utilisant la formule $\mathtt{a}^{-k} = \mathtt{R}^k$.

In [33]:
a_inverse = F("R")

# Trouver k tel que zeta = a^k
k = 0
power_a = F(1)

while power_a != zeta:
    power_a *= a
    k += 1

print(f"zeta = {zeta}, a^k = a^{k} = {power_a}")

# Calculer l'inverse de zeta comme a^(-k)
zeta_inverse_astuce = a_inverse ** k
print(f"L'inverse de zeta (calculé astucieusement) est {zeta_inverse_astuce}")

# Vérifier que le résultat est correct
assert zeta * zeta_inverse_astuce == F(1), f"Erreur: {zeta} * {zeta_inverse_astuce} != 1"


zeta = x, a^k = a^31 = x
L'inverse de zeta (calculé astucieusement) est W


### d) Solutions d'une équation polynomiale

<div class='alert-danger'> <strong> Alerte à la question! </strong> </div>

- Déterminer les racines dans $\mathbb{F}_{64}$ du polynôme $X^6 + X^4 + X^3 + X + 1 \in \mathbb{F}_2[X]$
- Expliquer comment cela permet de se convaincre que chaque élément de $\mathbb{F}_{64}$ peut s'écrire, de façon unique, sous la forme d'une expression de la forme

    $$ \boxed{b_5 \mathtt{a}^5 + b_4 \mathtt{a}^4 + b_3 \mathtt{a}^3 + b_2 \mathtt{a}^2 + b_1 \mathtt{a} + b_0} $$

    avec $(b_0, \ldots, b_5) \in \mathbb{F}_2^6$
- Quelle est l'écriture correspondante pour votre élément $\zeta$ ci-dessus ?

1) Pour déterminer les racines dans $\mathbb{F}_{64}$ du polynôme $X^6 + X^4 + X^3 + X + 1 \in \mathbb{F}2[X]$, nous allons évaluer le polynôme pour chaque élément de $\mathbb{F}{64}$ et vérifier si le résultat est le neutre additif (0).

In [34]:
def evaluate_polynomial(x):
    return x**6 + x**4 + x**3 + x + F(1)

roots = []

for element in F.elements[1:]:  # On exclut le neutre additif (0)
    if evaluate_polynomial(element) == F(0):
        roots.append(element)

print("Les racines du polynôme sont :")
for root in roots:
    print(root)


Les racines du polynôme sont :
a
c
o
r
W
!


2)Le fait que nous ayons trouvé les racines du polynôme $P(X) = X^6 + X^4 + X^3 + X + 1 \in \mathbb{F}2[X]$ dans $\mathbb{F}{64}$ montre que ce polynôme est un polynôme générateur minimal pour le groupe multiplicatif engendré par $\mathtt{a}$.

Le groupe multiplicatif engendré par $\mathtt{a}$ est un sous-groupe cyclique de l'ensemble des éléments non nuls de $\mathbb{F}_{64}$. Cela signifie que chaque élément de ce groupe peut être écrit comme une puissance de $\mathtt{a}$.

Puisque le polynôme $P(X)$ est de degré 6 et qu'il a des coefficients dans $\mathbb{F}_2$, cela implique que les exposants de $\mathtt{a}$ dans les expressions sous la forme $b_5 \mathtt{a}^5 + b_4 \mathtt{a}^4 + b_3 \mathtt{a}^3 + b_2 \mathtt{a}^2 + b_1 \mathtt{a} + b_0$ sont compris entre $0$ et $5$. Les coefficients $b_i$ appartiennent à $\mathbb{F}_2$, c'est-à-dire qu'ils peuvent être $0$ ou $1$.

Il y a $2^6 = 64$ combinaisons possibles de coefficients $(b_0, b_1, b_2, b_3, b_4, b_5)$. Comme il y a 64 éléments dans $\mathbb{F}{64}$, chaque élément de $\mathbb{F}{64}$ peut être représenté de manière unique par une expression de la forme $b_5 \mathtt{a}^5 + b_4 \mathtt{a}^4 + b_3 \mathtt{a}^3 + b_2 \mathtt{a}^2 + b_1 \mathtt{a} + b_0$ avec $(b_0, \ldots, b_5) \in \mathbb{F}_2^6$.

3)

In [35]:
a = F("a")
zeta_expr = F(0) * a**5 + F(1) * a**4 + F(1) * a**3 + F(0) * a**2 + F(0) * a + F(1)
print(f"zeta_expr = {zeta_expr}")


zeta_expr = x


In [36]:
combinations = [(b5, b4, b3, b2, b1, b0) for b5 in range(2) for b4 in range(2) for b3 in range(2) for b2 in range(2) for b1 in range(2) for b0 in range(2)]

for (b5, b4, b3, b2, b1, b0) in combinations:
    zeta_expr = F(b5) * a**5 + F(b4) * a**4 + F(b3) * a**3 + F(b2) * a**2 + F(b1) * a + F(b0)
    if zeta_expr == zeta:
        print(f"La combinaison qui donne zeta = x est : (b5, b4, b3, b2, b1, b0) = ({b5}, {b4}, {b3}, {b2}, {b1}, {b0})")


La combinaison qui donne zeta = x est : (b5, b4, b3, b2, b1, b0) = (0, 1, 1, 0, 0, 1)


## Partie 2 - chaînes de caractères et applications

Les éléments de $\mathbb{F}_{64}$ étant des caractères, on considérer une chaîne de caractères de longueur $n$ comme un élément de $\mathbb{F}_{64}^n$. Voici un début de classe permettant de considérer une chaîne de caractères (admissibles) comme une liste d'élémentsde $\mathbb{F}_{64}$.

In [37]:
class V(SageObject):
    
    # constructor 
    # s is a string of eligible characters 
    def __init__(self, s):
        
        self.coeffs = [F(c) for c in s]
        
    # representation of a V object 
    def __repr__(self):
        
        return "".join([c.__repr__() for c in self.coeffs])
    
    # result of V1 == V2 
    def __eq__(self, other):
        
        return self.coeffs == other.coeffs
   
    # length of s
    def __len__(self):
        
        return len(self.coeffs)
    
    # sum of two V objects
    def __add__(self, other):
        
        if len(self) != len(other):
            raise ValueError("Vectors must have same length")
        
        # create new V object to store the sum
        res = V("")
        
        # sum each coefficient of the two vectors
        for i in range(len(self)):
            res.coeffs.append(self.coeffs[i] + other.coeffs[i])
            
        return res

    # computes the checksum of the V object
    def checksum(self):
        s = F(0)
        for c in self.coeffs:
            s += c
        return s




### a) Somme de vecteurs

<div class='alert-danger'> <strong> Alerte à la question! </strong> </div>

Ajouter à la classe `V` une méthode `__add__` permettant de faire la somme **terme à terme** de deux chaînes de caractères de même longueur. Par exemple, on devrait avoir:
- `V("ceci") + V("cela") == V("00hg")`
- `V("CECI") + V("cela") == V("yKrK")` 
- `V("ceci") + V("CELA") == V("yKHu")`
- `V("CECI") + V("CELA") == V("00-?")` 

In [38]:
v1 = V("ceci")
v2 = V("cela")
v3 = V("CECI")
v4 = V("CELA")

print(v1 + v2)  # output: 00hg
print(v3 + v2)  # output: yKrK
print(v1 + v4)  # output: yKHu
print(v3 + v4)  # output: 00-?


00hg
yKrK
yKHu
00-?


### b) Chiffrement à clé secrète

On peut utiliser l'addition de chaînes de caractères ainsi définie pour garantir la confidentialité d'une chaîne de caractères donnée. Par exemple : soit le message clair `m` ci-dessous. 

In [39]:
m = V("Texte clair")

On peut le rendre inintelligible par un tiers en le masquant à l'aide d'une clé secrète `k` générée aléatoirement: 

In [40]:
k = V("kqaS%LAKVw'")

On obtient alors la chaîne incompréhensible suivante calculant la *somme* `c` de `m` et `k`: 

In [41]:
c = V("Hsz##pwPXqN")

<div class='alert-danger'> <strong> Alerte à la question! </strong> </div>

En utilisant les propriétés de $\mathbb{F}_{64}$: 
- indiquer comment déchiffrer le code si la clé est connu 
- déchiffrer le message `V("Kcgw&YCVFSl")` codé avec la même clé 

Dans l'anneau $\mathbb{F}_{64}$, chaque élément est son propre opposé parce que l'opération d'addition est définie comme une addition bit à bit (XOR) sur les identifiants des éléments. 

Le déchiffrement consiste donc simplement à calculer la somme entre la chaîne chiffrée et la clé secrète. En effet,puisque l'addition dans $\mathbb{F}_{64}$ est commutative et associative, on a :

$c + k =(m + k) + k = m + (k + k) = m + 0 = m$

Pour déchiffrer le message V("Kcgw&YCVFSl"), nous devons utiliser la même clé secrète k que celle utilisée pour chiffrer le message m. Nous calculons donc la somme entre V("Kcgw&YCVFSl") et k pour retrouver le message clair m :

In [42]:
# clé secrète
k = V("kqaS%LAKVw'")

# message chiffré
c = V("Kcgw&YCVFSl")

# déchiffrement
m = c + k

print(m) 

Oui bravo !


### c) Détection d'erreurs

On peut utiliser la structure algébrique mise sur les caractères pour *détecter* automatiquement des erreurs de cohérence dans une chaîne de caractères (erreur de transmission, media de stockage corrompu, ...). L'idée est toute simple: on ajoute une **somme de contrôle**.  

La version la plus simple consiste à fournir pour chaque chaîne de caractères la somme de tous ceux-ci.

<div class='alert-danger'> <strong> Alerte à la question! </strong> </div>

Implémenter une fonction pour calculer la somme de contrôle d'un élément de $\mathbb{F}_{64}^n$. Déterminer la somme de contrôle de la phrase `It's burger time!` 

Pour calculer la somme de contrôle d'un élément $v$ de $\mathbb{F}_{64}^n$, il suffit de faire la somme de tous les éléments de $v$ : $$checksum(v)=\sum_{i=1}^{n}v_i$$, ainsi dans la classe V ci-dessus, on ajoute la fonction suivante : 

```
# computes the checksum of the V object
def checksum(self):
    s = F(0)
    for c in self.coeffs:
        s += c
    return s
    
```

In [43]:
v = V("It's burger time!")
print(v.checksum())


I


<div class='alert-danger'> <strong> Alerte à la question! </strong> </div>

On peut détecter des erreurs si la somme de contrôle calculée ne correspond pas à la somme de contrôle attendue. Comprenez-le avec l'exemple ci-dessous. 

In [44]:
v = V("Ici y a-t-il une erreur ? C#mment le savez-vous ?")
s = F("Z") # expected checksum 

# compute the checksum of v and print it 
print(v.checksum())

# check if the computed checksum is equal to the expected one 
if v.checksum() == s:
    print("No error detected")
else:
    print("Error detected")


C
Error detected


Dans ce cas, on peut remarquer que la somme de contrôle attendue s est égale à l'élément F("Z"), mais que la somme de contrôle calculée ne correspond pas à cette valeur attendue. Par conséquent, on peut en déduire qu'il y a une erreur dans la chaîne de caractères v.

<div class='alert-danger'> <strong> Alerte à la culture! </strong> </div>

Notez que si, en plus de détecter la présence d'une erreur dans la chaîne, on sait à quel caractère elle s'est produite, on peut même la corriger automatiquement en lui ajoutant le scalaire approprié ; c'est l'idée de base des codes correcteurs d'erreurs (comme le [code de Reed-Solomon](https://en.wikipedia.org/wiki/Reed%E2%80%93Solomon_error_correction)) qui sont extensivement utilisés dans tous les systèmes de communication modernes.